In [342]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures


In [343]:
df= pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Housing.csv')
df.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [344]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [345]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

In [346]:
ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False,
).set_output(transform='pandas')


In [347]:
X = df.drop('price', axis=1)
y = df['price']

X_transformed = ct.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=24)



### Elastic Net Regression

In [348]:
# Elastic Net Regression
alphas = np.linspace(0.0001, 10, 20)  
l1 = np.linspace(0.0001, 1, 10)
scores = []

for a in alphas:
    for l in l1:
        er = ElasticNet(alpha=a, l1_ratio=l)
        er.fit(X_train, y_train)
        y_pred = er.predict(X_test)
        scores.append({
            'alpha': a,
            'l1_ratio': l,
            'score': r2_score(y_test, y_pred)
        })

df_scores = pd.DataFrame(scores)
df_scores.sort_values('score', ascending=False)

print(f'Best parameters: \nalpha: {df_scores.alpha.iloc[0]} \nl1_ratio: {df_scores.l1_ratio.iloc[0]} \nscore: {df_scores.score.iloc[0]}\n\n')

print(df_scores)

Best parameters: 
alpha: 0.0001 
l1_ratio: 0.0001 
score: 0.6246691191839193


       alpha  l1_ratio     score
0     0.0001    0.0001  0.624669
1     0.0001    0.1112  0.624671
2     0.0001    0.2223  0.624673
3     0.0001    0.3334  0.624675
4     0.0001    0.4445  0.624676
..       ...       ...       ...
195  10.0000    0.5556  0.306740
196  10.0000    0.6667  0.330276
197  10.0000    0.7778  0.368485
198  10.0000    0.8889  0.441074
199  10.0000    1.0000  0.624580

[200 rows x 3 columns]


In [349]:
elastic = ElasticNet(alpha=df_scores.alpha.iloc[0], l1_ratio=df_scores.l1_ratio.iloc[0])
elastic.fit(X_train, y_train)


for col, cof  in zip(list(X_test.columns), list(elastic.coef_)):
    print(f'{col}: {cof}')


lotsize: 3.6089487096622244
bedrooms: 1904.745632155252
bathrms: 14252.799010284163
stories: 6509.271287141077
garagepl: 5155.478681511246
driveway_yes: 7373.845572027081
recroom_yes: 3606.208192036937
fullbase_yes: 7304.743496882262
gashw_yes: 10859.74756012407
airco_yes: 12309.15116376799
prefarea_yes: 7656.582417338645
